### This file carries out orthogonal projection from a Cartesian point in space to an ellipsoid Earth.

Equatorial Radius $r_{e} =  6,378,136.6 $ $ (m)$

Polar Radius $r_{p} =   6,356,751.9 $ $ (m)$


Mathematical Expression of Earth in Cartesian coordinates:

$$ \frac{x^2}{{r_e}^2} + \frac{y^2}{{r_p}^2} + \frac{z^2}{{r_e}^2} = 1 $$


In [3]:
import numpy as np
import math
import matplotlib as plt

import utilities as ut

class Earth():
    def __init__(self, x=0, y=0, angle=0):
        
        '''
        Properties of the Earth and its position.
        '''

        self.re = 6378136.6
        self.rp = 6356751.9
        self.omega = 7.2921159 * 10**(-5)  

        self.x = x 
        self.y = y
        self.angle = angle

    def rotation_matrix(self):
        '''
        Returns the rotation matrix for the ellipse's rotation.
        '''
        a = math.cos(self.angle)
        b = math.sin(self.angle)
        return np.array([[a, -b], [b, a]])


    def distane_to_surface(self, x, tolerance = 1e-6, max_iterations = 1000):
        '''
        Finds the minimum distance between the specified point and the ellipse
        using Newton's method.

        NEEDS TO BE OPTIMISED FOR POLAR COORDINATES
        '''

        '''
        x comes in as a set of 3D polar coordinates, since we want it to be in cartesian 

        x1 - ellipse coordinate
        x2 - normalised satellite coordinates
        ''' 

        x = np.asarray(x)
        relevant_state = np.array([[x[0]], [x[1]]])
        x = ut.p_to_c(relevant_state)

        r = self.rotation_matrix()
        x2 = np.dot(r.T, x - [self.x, self.y])
        t = math.atan2(x2[1], x2[0])

        a = self.re
        b = self.rp

        if (x2[0] / a)**2 + (x2[1] / b)**2 < 1:
            ts = np.linspace(0, 2 * math.pi, 24, endpoint = False)
            xe = a * np.cos(ts)
            ye = b * np.sin(ts)
            delta = x2 - np.column_stack([xe, ye])
            t = ts[np.argmin(np.linalg.norm(delta, axis = 1))]
            
        iterations = 0
        error = tolerance
        errors = []
        ts = []
                
        while error >= tolerance and iterations < max_iterations:
            cost = math.cos(t)
            sint = math.sin(t)
            x1 = np.array([a * cost, b * sint])
            xp = np.array([-a * sint, b * cost])
            xpp = np.array([-a * cost, -b * sint])
            delta = x1 - x2
            dp = np.dot(xp, delta)
            dpp = np.dot(xpp, delta) + np.dot(xp, xp)

            t -= dp / dpp
            error = abs(dp / dpp)

            errors.append(error)
            ts.append(t)
            iterations += 1
        
        ts = np.array(ts)
        errors = np.array(errors)
        y = np.linalg.norm(x1 - x2)
        success = error < tolerance and iterations < max_iterations
        return dict(x = t, y = y, error = error, iterations = iterations, success = success, xs = ts,  errors = errors)



ImportError: cannot import name 'utilities' from 'utilities' (c:\Users\zhikh\Documents\GitHub\Satellite-Project\utilities.py)